In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException

import pandas as pd

import locale
from datetime import datetime, timedelta
import time

In [33]:
driver = webdriver.Firefox()

In [34]:
driver.get('https://www.sh-tourismus.de/veranstaltungskalender')

In [35]:
# Wait until the button is present
wait = WebDriverWait(driver, 10)

In [36]:
# Wait for the iframe to be present and switch to it
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'iframe[src*="tashwhl.infomaxnet.de"]'))
)
driver.switch_to.frame(iframe)

In [37]:
#while True:
for i in range(400): #TODO konfigurierbar, ca 300 events pro tag bisschen puffer, 10 events pro mal clicken laden also 30 mal klicken pro tag a 10 tage incl puffer müsste so ok sein, paar spätere mit rein
    try:
        button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, '-IMXEVENT-lazyLoadButton')))
        # Click the button
        button.click()
    except StaleElementReferenceException:
        # Handle the case where the element is no longer attached to the DOM
        continue
    except TimeoutException:
        # Handle the case where the element is not found within the timeout
        print("Button not found or not clickable within the timeout period.")
        break

KeyboardInterrupt: 

In [11]:
# Find all article elements representing events
#articles = driver.find_elements(By.CSS_SELECTOR, 'article.-IMXEVNT-listElement')

# Iterate through each article and extract information
for article in articles:
    # Extract the required information
    event_category = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__subline').text
    event_title = article.find_element(By.CSS_SELECTOR, 'span.-IMXEVNT-title').get_attribute('data-uppertitle')
    event_date_location = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__info').text

    # Print the extracted information
    print(f"Category: {event_category}")
    print(f"Title: {event_title}")
    print(f"Date and Location: {event_date_location}")
    #print(f"Description: {event_description}")
    print('-' * 40)

Category: Musikveranstaltungen
Title: INTERNATIONALER ORGELZYKLUS JOHANN SEBASTIAN BACH
Date and Location: 05.10.2024 / 00:00 Uhr / Lübecker Innenstadtkirchen, Lübeck
----------------------------------------
Category: Sonstiges
Title: QUARTIERSARBEIT DER VERBRAUCHERZENTRALE SCHLESWIG-HOLSTEIN
Date and Location: 05.10.2024 / 00:00 Uhr / Verbraucherzentrale Schleswig Holstein, Lübeck
----------------------------------------
Category: Sonstiges
Title: SICHT_FELDER FüR LüBECK
Date and Location: 05.10.2024 / 01:00 Uhr / Arbeitsgemeinschaft Kulturtreibhaus, Lübeck
----------------------------------------
Category: sonstige Seminare & Kurse
Title: FOTOAUSSTELLUNG „ÜBERGäNGE“ DES FOTOCLUBS NORDERSTEDT E.V.
Date and Location: 05.10.2024 / 05:00 - 22:00 Uhr / Stadtpark Norderstedt, Norderstedt
----------------------------------------
Category: Sonstiges
Title: LÜBECKER ÜBERGANGSGARTEN
Date and Location: 05.10.2024 / 06:00 - 23:00 Uhr / auf dem Koberg, Lübeck
-------------------------------------

In [38]:
# Find all article elements representing events
articles = driver.find_elements(By.CSS_SELECTOR, 'article.-IMXEVNT-listElement')

In [40]:
# List to store event data
events_data = []

# Iterate through each article and extract information
for article in articles:
    try:
        # Extract the required information
        event_category = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__subline').text
    except NoSuchElementException:
        event_category = 'N/A'

    try:
        event_title = article.find_element(By.CSS_SELECTOR, 'span.-IMXEVNT-title').get_attribute('data-uppertitle')
    except NoSuchElementException:
        event_title = 'N/A'

    try:
        event_date_location = article.find_element(By.CSS_SELECTOR, 'p.-IMXEVNT-listElement__text__info').text
    except NoSuchElementException:
        event_date_location = 'N/A'

    # Split the event_date_location into date, time, and location
    if event_date_location != 'N/A':
        parts = [x.strip() for x in event_date_location.split('/')]
    else:
        parts = []

    if len(parts) >= 3:
        date, time, location = parts[0], parts[1], ' / '.join(parts[2:])
    elif len(parts) == 2:
        date, time, location = parts[0], parts[1], ''
    else:
        date, time, location = 'N/A', 'N/A', 'N/A'

    # Split the location into location and city at the last comma
    if ',' in location:
        location, city = [x.strip() for x in location.rsplit(',', 1)]
    else:
        location, city = location, 'N/A'

    # Append the event data to the list
    events_data.append({
        'Category': event_category,
        'Title': event_title,
        'Date': date,
        'Time': time,
        'Location': location,
        'City': city
    })

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(events_data)

df

,Category,Title,Date,Time,Location,City
0,Musikveranstaltungen,INTERNATIONALER ORGELZYKLUS JOHANN SEBASTIAN BACH,05.10.2024,00:00 Uhr,Lübecker Innenstadtkirchen,Lübeck
1,Sonstiges,QUARTIERSARBEIT DER VERBRAUCHERZENTRALE SCHLES...,05.10.2024,00:00 Uhr,Verbraucherzentrale Schleswig Holstein,Lübeck
2,Sonstiges,SICHT_FELDER FüR LüBECK,05.10.2024,01:00 Uhr,Arbeitsgemeinschaft Kulturtreibhaus,Lübeck
3,sonstige Seminare & Kurse,FOTOAUSSTELLUNG „ÜBERGäNGE“ DES FOTOCLUBS NORD...,05.10.2024,05:00 - 22:00 Uhr,Stadtpark Norderstedt,Norderstedt
4,Sonstiges,LÜBECKER ÜBERGANGSGARTEN,05.10.2024,06:00 - 23:00 Uhr,auf dem Koberg,Lübeck
...,...,...,...,...,...,...
2603,Wochenmärkte,WOCHENMARKT MALZMüLLERWIESEN,24.10.2024,07:00 - 12:00 Uhr,Malzmüllerwiesen Itzehoe,Itzehoe
2604,sonstige Feste & Märkte,WOCHENMARKT TRAVEMüNDE,24.10.2024,07:30 - 13:00 Uhr,Wochenmarkt Travemünde,Travemünde
2605,sonstige Ausflüge & Führungen,"""BEI WIND UND WETTER""",24.10.2024,08:00 - 08:30 Uhr,Rückseite / Apsis der Klosterkirche,Cismar
2606,sonstige Seminare & Kurse,DAS POLNISCHE PLAKAT,24.10.2024,08:00 - 18:30 Uhr,Rathaus Reinbek,Reinbek


In [41]:
df.to_csv("Scraped_Events_Schleswig_Holstein")